In [1]:
!git clone https://github.com/tli9181991/NewsRiskClassificationwithGemma2.git

fatal: destination path 'NewsRiskClassificationwithGemma2' already exists and is not an empty directory.


In [2]:
!pip install -Uq bitsandbytes==0.46.0
!pip install -Uq peft==0.15.2
!pip install -Uq trl==0.18.2
!pip install -Uq accelerate==1.7.0
!pip install -Uq datasets==3.6.0
!pip install -Uq transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 72.9 MB/s eta 0:00:00


In [3]:
import os
import transformers
import torch
from datasets import Dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
from google.colab import userdata
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [5]:
def jsonstr2dict(json_str):
    json_str = json_str.replace("```", "")
    json_str = json_str.replace("\n", "")
    json_str = json_str.replace("json", "")

    return eval(json_str)

def dict2ans(rank_dict):
    item_list = list(rank_dict.items())
    ans_str = ""
    for risk, rank in item_list:
        ans_str += f"{risk}: {rank}\n"

    return ans_str

def jsonstr2ans(json_str):
    ans_dict = jsonstr2dict(json_str)
    return dict2ans(ans_dict)

In [6]:
df_train = pd.read_csv('/content/NewsRiskClassificationwithGemma2/rank_train.csv')
df_train.head(3)

,ticker,headline,description,answer
0,AAPL,Alibabas (BABA) New Qwen3 AI Models Now Compa...,Alibaba Group Holding Limited (NYSE:BABA) is o...,Strategic Risk: 3\nOperational Risk: 0\nFinanc...
1,AAPL,Apple updates the rules for its EU App Store b...,Apple on Thursday announced a series of update...,Strategic Risk: -3\nOperational Risk: -2\nFina...
2,AAPL,HSBC Just Flipped Bullish on Broadcom (AVGO)H...,Broadcom Inc. (NASDAQ:AVGO) is one of the 10 T...,Strategic Risk: 7\nOperational Risk: 0\nFinanc...


In [7]:
df_test = pd.read_csv('/content/NewsRiskClassificationwithGemma2/rank_test.csv')
df_test.head(3)

,ticker,headline,description,answer
0,AAPL,Apples (AAPL) Next Big AI Move? Analyst Sees ...,Apple Inc. (NASDAQ:AAPL) is one of the 10 Tren...,Strategic Risk: 10\nOperational Risk: 0\nFinan...
1,AAPL,Nvidia: How the chipmaker evolved from a gamin...,Heres a look at Nvidias path to where it is ...,Strategic Risk: -7\nOperational Risk: 0\nFinan...
2,AAPL,Apple (AAPL) Staring at 500 Fine in EU over A...,Apple Inc. (NASDAQ:AAPL) is one of the 12 best...,Strategic Risk: -3\nOperational Risk: -1\nFina...


In [8]:
risk_list = ['Strategic Risk',
 'Operational Risk',
 'Financial Risk',
 'Market Risk',
 'Technology Risk',
 'Credit Risk',
 'Legal Risk',
 'Political Risk',
 'Reputational Risk',
 'ESG Risk']

In [9]:
from datasets import Dataset
import torch

In [10]:
df_test["answer"].iloc[388] = "Strategic Risk: 3\nOperational Risk: 0\nFinancial Risk: 6\nMarket Risk: 2\nTechnology Risk: 0\nCredit Risk: 1\nLegal Risk: 0\nPolitical Risk: -2\nReputational Risk: 2\nESG Risk: 0\n"

/tmp/ipython-input-1541121604.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_test["answer"].iloc[388] = "Strategic Risk: 3\nOperational Risk: 0\nFinancial Risk: 6\nMarket Risk: 2\nTechnology Risk: 0\nCredit Risk: 1\nLegal Risk: 0\nPo

In [11]:
df_train["answer"].iloc[175] = "Strategic Risk: 0\nOperational Risk: 0\nFinancial Risk: 0\nMarket Risk: 0\nTechnology Risk: 0\nCredit Risk: 1\nLegal Risk: 0\nPolitical Risk: 0\n"

/tmp/ipython-input-3570781535.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_train["answer"].iloc[175] = "Strategic Risk: 0\nOperational Risk: 0\nFinancial Risk: 0\nMarket Risk: 0\nTechnology Risk: 0\nCredit Risk: 1\nLegal Risk: 0\nP

In [12]:
def preprocess_function(examples):
    # Combine headline and content
    text = [f"Ticker: {t}\nHeadline: {h}\nDescription: {c}" for t, h, c in zip(examples["ticker"], examples["headline"], examples["description"])]

    # Tokenize
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )

    # Parse answer into risk vector
    labels = []
    for ans in examples["answer"]:
        risk_dict = {}
        for line in ans.split("\n"):
            if not line.strip():
                continue
            risk, rank = line.split(": ")
            risk_dict[risk] = float(rank)
        # Build fixed-length vector
        vec = [risk_dict.get(r, 0.0) for r in risk_list]
        labels.append(vec)

    tokenized["labels"] = labels
    return tokenized

In [13]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [14]:
HF_TOKEN = userdata.get("HF_TOKEN")

In [15]:
model_id = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN, trust_remote_code=True, use_fast=False)

In [16]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
train_dataset = train_dataset.remove_columns(["ticker", "headline", "description", "answer"])

test_dataset = test_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.remove_columns(["ticker", "headline", "description", "answer"])

Map:   0%|          | 0/2630 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

In [ ]:
# for ans in df_train["answer"]:
#     risk_dict = {}
#     for line in ans.split("\n"):
#         if not line.strip():
#             continue

#         risk, rank = line.split(": ")
#         risk_dict[risk] = float(rank)

In [ ]:
# for i in range(len(df_train["answer"])):
#     ans = df_train["answer"].iloc[i]
#     if 'AMZN' in ans:
#       print(i)
#       break

175


In [ ]:
# df_train["answer"].iloc[175] = "Strategic Risk: 0\nOperational Risk: 0\nFinancial Risk: 0\nMarket Risk: 0\nTechnology Risk: 0\nCredit Risk: 1\nLegal Risk: 0\nPolitical Risk: 0\n"

/tmp/ipython-input-3570781535.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_train["answer"].iloc[175] = "Strategic Risk: 0\nOperational Risk: 0\nFinancial Risk: 0\nMarket Risk: 0\nTechnology Risk: 0\nCredit Risk: 1\nLegal Risk: 0\nP

In [ ]:
# class RiskDataset(Dataset):
#     def __init__(self, df, tokenizer, risk_list, max_len=512):
#         self.df = df
#         self.tokenizer = tokenizer
#         self.max_len = max_len
#         self.risk_list = risk_list

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         item = self.df.iloc[idx]
#         text = f"Headline: {item['headline']}\nContent: {item['content']}"
#         risk_dict = {}
#         for risk_str in item['answer'].split('\n'):
#             if not risk_str.strip():
#                 continue
#             risk, rank = risk_str.split(': ')
#             risk_dict[risk] = float(rank)

#         label = []
#         for risk in self.risk_list:
#             label.append(risk_dict.get(risk, 0.0))

#         label = torch.tensor(label, dtype=torch.float)  # -10 to 10

#         inputs = self.tokenizer(
#             text,
#             truncation=True,
#             max_length=self.max_len,
#             padding="max_length",
#             return_tensors="pt"
#         )
#         return {
#             "input_ids": inputs["input_ids"].squeeze(0),
#             "attention_mask": inputs["attention_mask"].squeeze(0),
#             "labels": label
#         }

In [ ]:
# dataset_train = RiskDataset(df_train, tokenizer, risk_list)
# dataset_test = RiskDataset(df_test, tokenizer, risk_list)

In [17]:
from transformers import AutoModelForCausalLM
import torch.nn as nn

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load base Gemma 2B
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             token=HF_TOKEN)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
model.lm_head = nn.Linear(model.config.hidden_size, len(risk_list), bias=True)
model.config.vocab_size = 1

model.lm_head.to(next(model.parameters()).device)

Linear(in_features=2048, out_features=10, bias=True)

In [19]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (n

In [20]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # adapt attention projections
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [21]:
model = get_peft_model(model, lora_config)

In [52]:
def compute_loss(model, inputs, return_outputs=False, num_items_in_batch=None):
    # Handle case where inputs might be a dict or tensor
    if isinstance(inputs, dict):
        labels = inputs["labels"]
        inputs = {k: v for k, v in inputs.items() if k != "labels"}
    else:
        raise ValueError(f"Expected inputs to be dict, got {type(inputs)}")

    outputs = model(**inputs)

    logits = outputs.logits
    if logits.ndim > 1 and logits.shape[-1] == 1:
        logits = logits.squeeze(-1)

    # Ensure labels are on same device
    labels = labels.to(logits.device)

    loss_fn = nn.MSELoss()
    loss = loss_fn(logits, labels)

    return (loss, outputs) if return_outputs else loss

In [53]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch"
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    # label_names=["labels"],
    report_to="none"
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-3101714075.py, line 11)

In [54]:
from transformers import DataCollatorWithPadding
collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")

In [64]:
# from transformers import Trainer

# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
#         # Handle case where inputs might be a dict or tensor
#         if isinstance(inputs, dict):
#             labels = inputs["labels"]
#             inputs = {k: v for k, v in inputs.items() if k != "labels"}
#         else:
#             raise ValueError(f"Expected inputs to be dict, got {type(inputs)}")

#         outputs = model(**inputs)

#         logits = outputs.logits
#         if logits.ndim > 1 and logits.shape[-1] == 1:
#             logits = logits.squeeze(-1)

#         # Ensure labels are on same device
#         labels = labels.to(logits.device)

#         loss_fn = nn.MSELoss()
#         loss = loss_fn(logits, labels)

#         return (loss, outputs) if return_outputs else loss

In [73]:
def msn_loss(outputs, labels, num_items_in_batch=None):

    logits = outputs.logits
    if logits.ndim > 1 and logits.shape[-1] == 1:
        logits = logits.squeeze(-1)

    # Ensure labels are on same device
    labels = labels.to(logits.device)

    loss_fn = nn.MSELoss()
    print(logits.shape)
    print(labels.shape)
    loss = loss_fn(logits, labels)

    return loss

In [74]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_loss_func= msn_loss,
    data_collator=collator
)

/tmp/ipython-input-1772702770.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [75]:
trainer.train()

torch.Size([1, 512, 10])
torch.Size([1, 10])


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([1, 512, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
train_dataset

Dataset({
    features: ['answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2630
})

In [45]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2630
})

In [47]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=2,
    collate_fn=collator
)

In [49]:
batch = next(iter(train_dataloader))

In [51]:
batch.pop('labels')

tensor([[ 3.,  0.,  0.,  4.,  8.,  0.,  0., -2.,  3.,  0.],
        [-3., -2.,  2., -2.,  0.,  0.,  3., -2., -4.,  1.]])

In [76]:
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch

class BertForRiskRegression(nn.Module):
    def __init__(self, base_model_name="bert-base-uncased", num_outputs=10):
        super().__init__()
        self.bert = AutoModel.from_pretrained(base_model_name)
        self.regression_head = nn.Linear(self.bert.config.hidden_size, num_outputs)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # BERT outputs: last_hidden_state, pooler_output
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]  # CLS token representation
        logits = self.regression_head(pooled)     # [batch, num_outputs]

        loss = None
        if labels is not None:
            loss = nn.MSELoss()(logits, labels)

        return {"loss": loss, "logits": logits}

In [77]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [78]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [79]:
def preprocess_function(examples):
    # Combine headline and content
    text = [f"Ticker: {t}\nHeadline: {h}\nDescription: {c}" for t, h, c in zip(examples["ticker"], examples["headline"], examples["description"])]

    # Tokenize
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )

    # Parse answer into risk vector
    labels = []
    for ans in examples["answer"]:
        risk_dict = {}
        for line in ans.split("\n"):
            if not line.strip():
                continue
            risk, rank = line.split(": ")
            risk_dict[risk] = float(rank)
        # Build fixed-length vector
        vec = [risk_dict.get(r, 0.0) for r in risk_list]
        labels.append(vec)

    tokenized["labels"] = labels
    return tokenized

In [80]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
train_dataset = train_dataset.remove_columns(["ticker", "headline", "description", "answer"])

test_dataset = test_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.remove_columns(["ticker", "headline", "description", "answer"])

Map:   0%|          | 0/2630 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

In [90]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./BertRiskRegression",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    learning_rate=5e-5,
    fp16=True,
    eval_strategy="steps",
    save_strategy="epoch",
    logging_steps=10,
    report_to="none"  # disable wandb
)

In [91]:
model = BertForRiskRegression(num_outputs=len(risk_list))

In [92]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-3688628395.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [93]:
trainer.train()

Step,Training Loss,Validation Loss
10,9.862900,11.115836
20,10.315300,10.965605
30,11.059000,10.683138
40,10.865700,10.042440
50,10.025400,9.986452
60,9.583500,9.322371
70,11.093200,9.141622
80,9.157500,8.714364
90,8.531800,8.628566
100,8.388200,9.099256


TrainOutput(global_step=1650, training_loss=4.072445548086455, metrics={'train_runtime': 1733.0914, 'train_samples_per_second': 15.175, 'train_steps_per_second': 0.952, 'total_flos': 0.0, 'train_loss': 4.072445548086455, 'epoch': 10.0})